In [1]:
import requests
import pandas as pd
import os
import datetime as dt
import json
import locale
import numpy as np
import matplotlib.pyplot as plt
import pyodbc
from sqlalchemy import create_engine
from sqlalchemy import create_engine, text
import difflib
from dateutil.parser import parse
import glob # Para leer archivos de una carpeta
import time
from dateutil.parser import parse
import calendar
import warnings
import mysql.connector
import glob # Para leer archivos de una carpeta
import pyautogui as pg
import time
import random
import webbrowser as web 
import re
import urllib.parse  # Importar para codificar mensajes correctamente
# script_principal.py
from actualizarRegistros import DatabaseConnection, actualizar_registros, actualizar_recurrencia
import pywhatkit as kit

# Función para enviar imagen por WhatsApp usando pywhatkit
def enviar_imagen(numero, image_path, caption):
    kit.sendwhats_image(numero, image_path, caption)


warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)


# Configurar locale para español (Perú)
locale.setlocale(locale.LC_TIME, 'es_PE.UTF-8')

# Variables PERIODO ABRIL 2024
fecha = '202410'


# Crear la fecha de inicio del mes concatenando con '01'
fecha_inicio = f"{fecha}01"
fecha_inicio_dt = pd.to_datetime(fecha_inicio, format='%Y%m%d')

usuario = 'azaer'

# Calcular el último día del mes
anio = int(fecha[:4])
Mes = int(fecha[4:6])
ultimo_dia = calendar.monthrange(anio, Mes)[1]

# Crear la fecha de fin del mes
fecha_fin = f"{fecha}{ultimo_dia}"
fecha_fin_dt = pd.to_datetime(fecha_fin, format='%Y%m%d')

print(f"Fecha de inicio: {fecha_inicio_dt}")
print(f"Fecha de fin: {fecha_fin_dt}")

# Convertir las fechas a cadenas en formato YYYY-MM-DD para usar en SQL
fecha_inicio_str = fecha_inicio_dt.strftime('%Y-%m-%d')
fecha_fin_str = fecha_fin_dt.strftime('%Y-%m-%d')

# Obtener la fecha actual
fecha_hoy = dt.datetime.now()

# Calcular cuántos días faltan para el próximo sábado (weekday() devuelve 0 para lunes y 6 para domingo)
dias_para_sabado = (5 - fecha_hoy.weekday()) % 7
# Si hoy ya es sábado, queremos el próximo sábado (no hoy)
if dias_para_sabado == 0:
    dias_para_sabado = 0

# Calcular la fecha del próximo sábado
fecha_sabado = fecha_hoy + dt.timedelta(days=dias_para_sabado)

print(fecha_sabado.strftime("%d de %B"))
evento_huaycan = 'Noche de Candys con Plan B'
evento_mala = 'Farandula repartera con DJ Jacko'
link_huaycan = 'www.beringbou.com'
link_mala = 'www.beringbousur.com'
imagen_huaycan = r"C:\Users\azaer\Documents\Diego\Bering\12-10\imagen_huaycan.jpg"


# Mes a partir del fecha
mes = dt.datetime.strptime(fecha, '%Y%m').strftime('%B').upper()
Ruta = rf'C:\Users\{usuario}\Documents\Diego\Bering'
Documentos = rf'C:\Users\{usuario}\Documents'
Descargas = rf'C:\Users\{usuario}\Downloads'


# Datos de conexión
db_config = {
    'server': r'database',
    'database': 'database',
    'username': 'username',
    'password': 'password'
}

# Crear instancia de la conexión
db = DatabaseConnection(**db_config)

# Conectar a la base de datos
db.connect()

# Actualizar registros y recurrencia
actualizar_registros(db)

# Cerrar la conexión
db.close()

actualizar_recurrencia(db)


Fecha de inicio: 2024-10-01 00:00:00
Fecha de fin: 2024-10-31 00:00:00
12 de Octubre
Conexión exitosa a la base de datos.
No hay nuevos registros para insertar.
Error ejecutando la consulta: Attempt to use a closed cursor.
RecurrenciaDNI_CEL actualizada.


c:\Users\azaer\Documents\Diego\Bering\actualizarRegistros.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, self.conn)
c:\Users\azaer\Documents\Diego\Bering\actualizarRegistros.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nuevos_registros.drop(columns=['fecha_registro'], inplace=True)


: 

In [4]:
# Abrir archivo que contenga "evento*.csv" en la carpeta de descargas
archivos = glob.glob(f"{Descargas}\\evento.csv")

# Leer el archivo
df = pd.read_csv(archivos[0], sep=';', encoding='latin1', dtype=str)

#Rellenar columna dni con ceros hasta 8 dígitos
df['dni'] = df['dni'].str.zfill(8).str.strip().str[0:8]

# Convertir la columna 'fecha' a formato datetime (asegúrate de que tiene el formato correcto)
df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d', errors='coerce')

#Eliminar Duplicados donde DNI + Numero
df = df.drop_duplicates(subset=['dni', 'numero'], keep='last')


#Eliminar vacios de Fecha
df = df.dropna(subset=['fecha', 'dni', 'evento', 'nombre'])

# Crear Columna Cumpleaños en Formato 14Octubre
df['Cumpleaños'] = df['fecha'].dt.strftime('%d%B')

# Crear una columna 'Fecha_Cumpleaños' con el año actual para evaluar los cumpleaños de este año
df['Fecha_Cumpleaños'] = pd.to_datetime(df['Cumpleaños'] + str(fecha_hoy.year), format='%d%B%Y', errors='coerce')

# Calcular la diferencia de días entre hoy y la fecha de cumpleaños
df['Diferencia_Dias'] = (df['Fecha_Cumpleaños'] - fecha_sabado).dt.days

# Calcular las veces que se repite el dni por promotor en el df
df['RecurrenciaDNI'] = df.groupby(['dni', 'promotor'])['dni'].transform('count')
df['RecurrenciaCel'] = df.groupby(['numero', 'promotor'])['numero'].transform('count')

# Columna Recurrencia concatenando DNI y Celular con "|"
df['RecurrenciaDNI_CEL'] = df['RecurrenciaDNI'].astype(str) + "|" + df['RecurrenciaCel'].astype(str)

# Eliminar columnas RecurrenciaDNI y RecurrenciaCel
df = df.drop(columns=['RecurrenciaDNI', 'RecurrenciaCel'])

# Añadir Columna No Apto para mensaje promocional si la diferencia de días es -6 a 6
df['No_Apto'] = np.where((-6 <= df['Diferencia_Dias']) & (df['Diferencia_Dias'] <= 6), 'NO', 'SI')

#Si se han registrado al Evento = 'SAB 12.OCTUBRE | NOCHE DE CANDYS - PLAN B' entonces No_Apto = 'NO'
df.loc[df['evento'] == 'SAB 12.OCTUBRE | NOCHE DE CANDYS - PLAN B', 'No_Apto'] = 'NO'

#Ordenar Dataframe priorizando los Fecha_registro más antiguo
df = df.sort_values(by=['fecha_registro'], ascending=True)

#Filtrar solo los que son aptos para mensaje promocional
df = df[df['No_Apto'] == 'SI']


# Añadir columnas 'Mensaje_Enviado' y 'Fecha_Mensaje_Envio' al DataFrame
df['Mensaje_Enviado'] = 'NO'
df['Fecha_Mensaje_Envio'] = None

# Si promotor es "Promotor Huaycan", entonces Columna Discoteca debe ser "Huaycan" caso contrario "Mala" en una sola línea
df['Discoteca'] = np.where(df['promotor'] == 'Promotor Huaycan', 'Huaycan', 'Mala')

# Función para generar el mensaje personalizado
def generar_mensaje(nombre, promotor):
    if promotor == "Promotor Huaycan":
        mensaje = (
            f"Hola {nombre}, te saluda el *Tío Bering*, este {fecha_sabado.strftime("%d de %B")} tenemos el evento especial {evento_huaycan} 🎉 y queremos invitarte a que disfrutes con nosotros\n"
            f"Te ofrecemos entradas Free si te registras en el link {link_huaycan} y muestras el QR en Puerta\n"
            "Además, estamos ofreciendo una botella sorpresa GRATIS si eres de los 50 primeros\n"
            "Pasa la voz🍻"
        )
    else:
        mensaje = (
            f"Hola {nombre}, te saluda el *Tío Bering*, este {fecha_sabado.strftime("%d de %B")} tenemos el evento especial {evento_mala} 🎉 y queremos invitarte a que disfrutes con nosotros.%0A"
            f"Te ofrecemos entradas Free si te registras en el link {link_mala} y muestras el QR en Puerta%0A"
            "Además, estamos ofreciendo una botella sorpresa GRATIS si eres de los 50 primeros%0A"
            "Pasa la voz🍻"
        )
    return mensaje


# Iterar sobre contactos (solo nombre, número y promotor) solo si promotor = 'Promotor Huaycan'
for index, row in df.iterrows():
    if row['promotor'] != 'Promotor Huaycan':
        continue
    nombre = row['nombre']
    numero = '+51' + row['numero']
    promotor = row['promotor']
    
    # Generar mensaje personalizado
    mensaje = generar_mensaje(nombre, promotor)

    #Enviar Mensaje instantaneo de imagen a WhatsApp
    kit.sendwhats_image(numero, imagen_huaycan, mensaje, 20)

    time.sleep(random.randint(1, 5))


#Conectarme a la base de datos para guardar los mensajes enviados
# Conectar a la base de datos

# Crear instancia de la conexión
db = DatabaseConnection(**db_config)

# Conectar a la base de datos
db.connect()

# Generar los enlaces de WhatsApp solo para las personas que tienen mensaje
df['Link_WhatsApp'] = "https://web.whatsapp.com/send?phone=51" + df['numero'] + "&text=" + df['Mensaje_Personalizado']

# Reemplazar caracteres especiales para el formato URL
df['Link_WhatsApp'] = df['Link_WhatsApp'].str.replace(" ", "%20")

#Si Mensaje Personalizado está vacío, entonces Link_WhatsApp también debe estar vacío
df.loc[df['Mensaje_Personalizado'] == "", 'Link_WhatsApp'] = ""

#Eliminar Columna Mensaje Personalizado y Fecha_Cumpleaños
df = df.drop(columns=['Mensaje_Personalizado', 'Fecha_Cumpleaños'])

#Realizar consulta en la base de datos para obtener los mensajes enviados y datos actualizados

# Ejecutar la consulta y guardar los datos en un DataFrame
consulta_sql = "SELECT * FROM MensajesWhatsApp"  # Reemplaza con la consulta que necesites
df_sql = db.fetch_data(consulta_sql)


#Eliminar duplicados en df_sql segun id
df_sql = df_sql.drop_duplicates(subset='id', keep='last')

#Cruzar df_sql con df para obtener Link_WhatsApp
df = df_sql.merge(df[['dni','Link_WhatsApp']], on='dni', how='left')

#Eliminar duplicados en df
df = df.drop_duplicates(subset='id', keep='last')

#Si está abierto, cierra el archivo
os.system(f"taskkill /f /im EXCEL.EXE")

#Esperar 5 segundos
time.sleep(5)

# Exportar el archivo con los enlaces de WhatsApp en formato xlsx
df.to_excel(f"{Ruta}\\Enlaces_WhatsApp_{mes}.xlsx", index=False)


# Abrir el archivo con los enlaces de WhatsApp
os.system(f"start EXCEL.EXE {Ruta}\\Enlaces_WhatsApp_{mes}.xlsx")

print("Proceso terminado")
